## 1.1 — Environment Setup

In [ ]:
# Install required packages
!pip install -q kaggle "tensorflow>=2.17.0" gradio matplotlib

# Python imports
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 1.2 — Kaggle Dataset Download

In [ ]:
# Upload kaggle.json API key
from google.colab import files
files.upload()  # Upload kaggle.json

# Set up Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset if not already present
if not os.path.exists("/content/new-plant-diseases-dataset"):
    !kaggle datasets download -d vipoooool/new-plant-diseases-dataset -p /content
    !unzip -q /content/new-plant-diseases-dataset.zip -d /content/new-plant-diseases-dataset

!ls -lah /content/new-plant-diseases-dataset | sed -n '1,200p'

## 1.3 — Configuration

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5
N_LAST_LAYERS = 10   # Unfreeze last 10 layers for fine-tuning
SEED = 1337
NUM_CLASSES = 38

train_dir = "/content/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_dir = "/content/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid"

# Verify paths
for p in [train_dir, valid_dir]:
    if not os.path.exists(p):
        print(f"ERROR: path not found: {p}")
        sys.exit(1)

print("train_dir:", train_dir)
print("valid_dir:", valid_dir)

## 1.4 — Data Exploration & Visualization

In [ ]:
import random
from IPython.display import Image, display

def show_sample_images(base_path, num_classes=5, images_per_class=3):
    """Display random sample images from the dataset"""
    class_names = os.listdir(base_path)
    random_classes = random.sample(class_names, min(num_classes, len(class_names)))

    fig, axes = plt.subplots(num_classes, images_per_class, figsize=(12, 3*num_classes))
    fig.suptitle('Sample Images from Dataset', fontsize=16)

    for i, class_name in enumerate(random_classes):
        class_path = os.path.join(base_path, class_name)
        images = os.listdir(class_path)
        random_images = random.sample(images, min(images_per_class, len(images)))

        for j, img_name in enumerate(random_images):
            img_path = os.path.join(class_path, img_name)
            img = plt.imread(img_path)
            axes[i, j].imshow(img)
            axes[i, j].axis('off')
            if j == 0:
                axes[i, j].set_title(class_name.replace('___', '\n'), fontsize=10)

    plt.tight_layout()
    plt.show()

show_sample_images(train_dir)

## 1.5 — Dataset Statistics

In [ ]:
def count_images(directory):
    """Count total images and images per class"""
    total_images = 0
    class_counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            num_images = len(os.listdir(class_path))
            class_counts[class_name] = num_images
            total_images += num_images
    return total_images, class_counts

train_total, train_counts = count_images(train_dir)
valid_total, valid_counts = count_images(valid_dir)

print(f"Training images: {train_total}")
print(f"Validation images: {valid_total}")
print(f"Number of classes: {len(train_counts)}")
print(f"\nClass distribution (first 10):")
for i, (class_name, count) in enumerate(list(train_counts.items())[:10]):
    print(f"  {class_name}: {count} images")

## 1.6 — Data Generators (Preprocessing & Augmentation)

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Training generator with LIGHT augmentation (dataset already augmented)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='reflect'
)

# Validation generator — NO augmentation
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

valid_gen = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

## 1.7 — Load Pre-trained MobileNetV2 & Configure Transfer Learning

In [ ]:
from tensorflow.keras.applications import MobileNetV2

# Load pretrained MobileNetV2 backbone
base_model = MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

print("MobileNetV2 loaded successfully!")
print(f"Total layers in base model: {len(base_model.layers)}")

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze last N layers for fine-tuning
if N_LAST_LAYERS > 0:
    for layer in base_model.layers[-N_LAST_LAYERS:]:
        layer.trainable = True

# Count parameters
trainable_count = sum([tf.size(w).numpy() for w in base_model.trainable_weights])
non_trainable_count = sum([tf.size(w).numpy() for w in base_model.non_trainable_weights])

print(f"Trainable parameters: {trainable_count:,}")
print(f"Non-trainable parameters: {non_trainable_count:,}")
print(f"Unfrozen last {N_LAST_LAYERS} layers for fine-tuning")

## 1.8 — Build Complete Model Architecture

In [ ]:
# Build the complete model
inputs = keras.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)  # Frozen BatchNorm layers
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.35)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = keras.Model(inputs, outputs, name="mobilenetv2_plant_disease_classifier")
model.summary()

## 1.9 — Compile the Model

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),  # Low LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully!")
print("Optimizer: Adam (lr=1e-4)")
print("Loss function: Categorical Crossentropy")
print("Metrics: Accuracy")

## 1.10 — Setup Training Callbacks

In [ ]:
callbacks = [
    # Save best model based on validation accuracy
    keras.callbacks.ModelCheckpoint(
        "/content/mobilenetv2_best.keras",
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    # Reduce learning rate when validation loss plateaus
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1
    ),
    # Stop training if no improvement
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=6,
        restore_best_weights=True,
        verbose=1
    )
]

print("Callbacks configured:")
print("  1. ModelCheckpoint - Saves best model")
print("  2. ReduceLROnPlateau - Adjusts learning rate")
print("  3. EarlyStopping - Prevents overfitting")

## 1.11 — Train the Model

In [ ]:
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=valid_gen,
    callbacks=callbacks
)

## 1.12 — Visualize Training History

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history.get('accuracy', []), label='train_accuracy')
plt.plot(history.history.get('val_accuracy', []), label='val_accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history.get('loss', []), label='train_loss')
plt.plot(history.history.get('val_loss', []), label='val_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.title('Loss')

plt.show()

## 1.13 — Model Evaluation

In [ ]:
val_loss, val_acc = model.evaluate(valid_gen)
print(f"Validation loss: {val_loss:.4f}, accuracy: {val_acc:.4f}")

## 1.14 — Save Final Model

In [ ]:
final_path = "/content/mobilenetv2_final.keras"
model.save(final_path)
print("Saved final model to:", final_path)

## 1.15 — Save Class Names

In [ ]:
import json

class_names = list(train_gen.class_indices.keys())
print(f"Total classes: {len(class_names)}")
print("First 5 classes:", class_names[:5])

# Save class names to JSON
with open('/content/class_names.json', 'w') as f:
    json.dump(class_names, f)
print("Class names saved to class_names.json")